In [5]:
import requests
from xml.etree import ElementTree
import pandas as pd
from datetime import datetime

# Set pandas options to display all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Base URL for PubMed E-utilities API
base_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'

# Current date and January 1 of five years ago
current_date = datetime.now().strftime('%Y/%m/%d')
five_years_ago_date = datetime(datetime.now().year - 5, 1, 1).strftime('%Y/%m/%d')

# Your query
query = f'Dyrk1A kinase inhibitor AND ("{five_years_ago_date}"[PDAT] : "{current_date}"[PDAT])'

# Create an empty DataFrame to store the results
df = pd.DataFrame(columns=['Title', 'URL', 'Authors', 'Journal', 'Publication Year'])

# Send a GET request to the ESearch utility to search for the query in PubMed
response = requests.get(f'{base_url}esearch.fcgi', params={
    'db': 'pubmed',
    'term': query,
    'retmode': 'xml'
})

# If the request is successful, parse the XML response
if response.status_code == 200:
    root = ElementTree.fromstring(response.text)
    # Get the list of PubMed IDs (PMIDs) from the response
    id_list = [id_elem.text for id_elem in root.iter('Id')]

    # For each PMID, send a GET request to the ESummary utility to retrieve the article details
    for pmid in id_list:
        response = requests.get(f'{base_url}esummary.fcgi', params={
            'db': 'pubmed',
            'id': pmid,
            'retmode': 'xml'
        })
        if response.status_code == 200:
            root = ElementTree.fromstring(response.text)
            # Extract the title, URL, authors, journal, and publication year for each article and add them to the DataFrame
            for docsum in root.iter('DocSum'):
                title = docsum.find('Item[@Name="Title"]').text
                url = f'https://pubmed.ncbi.nlm.nih.gov/{pmid}/'
                authors = docsum.find('Item[@Name="Authors"]').text if docsum.find('Item[@Name="Authors"]') is not None else 'N/A'
                journal = docsum.find('Item[@Name="FullJournalName"]').text if docsum.find('Item[@Name="FullJournalName"]') is not None else 'N/A'
                pub_year = docsum.find('Item[@Name="PubDate"]').text.split()[0] if docsum.find('Item[@Name="PubDate"]') is not None else 'N/A'
                df = df.append({'Title': title, 'URL': url, 'Authors': authors, 'Journal': journal, 'Publication Year': pub_year}, ignore_index=True)

# Display the DataFrame
print(df)

# Save the DataFrame as a CSV file
df.to_csv('literature.csv', index=False)


<ipython-input-5-0b1cabd9c640>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Title': title, 'URL': url, 'Authors': authors, 'Journal': journal, 'Publication Year': pub_year}, ignore_index=True)
<ipython-input-5-0b1cabd9c640>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Title': title, 'URL': url, 'Authors': authors, 'Journal': journal, 'Publication Year': pub_year}, ignore_index=True)
<ipython-input-5-0b1cabd9c640>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Title': title, 'URL': url, 'Authors': authors, 'Journal': journal, 'Publication Year': pub_year}, ignore_index=True)
<ipython-input-5-0b1cabd9c640>:52: FutureWarning: The frame.append method is deprecated 

                                                Title  \
0   On a path toward a broad-spectrum anti-viral: ...   
1   Leucettinibs, a Class of DYRK/CLK Kinase Inhib...   
2   Azaindole derivatives as potential kinase inhi...   
3   Urolithin A exhibits a neuroprotective effect ...   
4   A genome-wide screen reveals that Dyrk1A kinas...   
5   An inhibitor with GSK3β and DYRK1A dual inhibi...   
6   Function and inhibition of DYRK1A: Emerging ro...   
7   Development of Novel Fluorinated Polyphenols a...   
8   Effects of Monoamino-Oxidase-A (MAO-A) Inhibit...   
9   A bivalent β-carboline derivative inhibits mac...   
10  Inhibition of CDK4/6 regulates AD pathology, n...   
11  Silmitasertib (CX-4945), a Clinically Used CK2...   
12  Comparative Efficacy and Selectivity of Pharma...   
13  Rational design and appraisal of selective Cdc...   
14  Disruption of DYRK1A-induced hyperphosphorylat...   
15  New insights into the regulation of Cystathion...   
16  FRTX-02, a selective and po

<ipython-input-5-0b1cabd9c640>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Title': title, 'URL': url, 'Authors': authors, 'Journal': journal, 'Publication Year': pub_year}, ignore_index=True)
